# Import

In [1]:
from collections import Counter
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
#import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
directory = 'D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/'
font_path = directory + 'NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

In [2]:
import time
from tqdm import tqdm

# Load Data

In [3]:
dev_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/dev.users'
dev_users_list = []
with open(dev_users_path, 'r') as fr:
    lines = fr.readlines()
    dev_users_list = lines
    del lines
for i in range(len(dev_users_list)):
    dev_users_list[i] = dev_users_list[i].replace('\n','')
    
test_users_path ='D:/ANACONDA/envs/tf-gpu/code/NLP/kakao/data/predict/predict/test.users'
test_users_list = []
with open(test_users_path, 'r') as fr:
    lines = fr.readlines()
    test_users_list = lines
    del lines
for i in range(len(test_users_list)):
    test_users_list[i] = test_users_list[i].replace('\n','')

print(len(dev_users_list))
print(len(test_users_list))

3000
5000


In [ ]:
- FEB_top_30000 : 2월 1일 ~ 3월 1일 읽은 글 top 30000
- TOTAL_top_30000 : 10월 1일 ~ 3월 1일까지 읽은 글 top 30000

- FEB_top_58647 : 2월 14일 ~ 3월 1일간 읽은 글 best
    
- FEB_MAR_article : 2월 1일 ~ 3월 까지 등록된 글
- FEB_MAR_article_2 : 2월 14일 ~ 3월 1일 등록된 글
- MAR_article_1 : 3월 1일 ~ 3월 7일 등록된 글
- MAR_article_2 : 3월 7일 ~ 3월 14일 등록된 글 
 *기간은 왼쪽 처음은 닫힌 구간, 오른쪽 끝은 열린 구간

In [4]:
FEB_top_58647 = pd.read_csv(directory+'FEB_top_58647.csv')
FEB_top_58647 = FEB_top_58647.drop('Unnamed: 0',1)
FEB_MAR_article_2 = pd.read_csv(directory+'FEB_MAR_article_2.csv')
FEB_MAR_article_2 = FEB_MAR_article_2.drop('Unnamed: 0',1)
MAR_article_1 = pd.read_csv(directory+'MAR_article_1.csv')
MAR_article_1 = MAR_article_1.drop('Unnamed: 0',1)
MAR_article_2 = pd.read_csv(directory+'MAR_article_2.csv')
MAR_article_2 = MAR_article_2.drop('Unnamed: 0',1)

print('20190723 update')
print(len(FEB_top_58647))
print(len(FEB_MAR_article_2)) # 이건 필요가 없을 수도 있겠다 어차피 안 팔릴 글은 안 읽었을테니
print(len(MAR_article_1))
print(len(MAR_article_2))


20190723 update
58648
11641
4654
5322


# 읽은 글 체크

In [ ]:
- article_seen_by_testuser

In [5]:
with open(directory + 'article_by_testuser.json') as f:
    article_seen_by_testuser = json.load(f)

## following list

In [6]:
users = pd.read_json(directory + '/users.json', lines=True)

In [7]:
import time
st = time.time()
users_following_lists = [] 
for i in range(5000):
    uid = test_users_list[i]
    ufl = list(users[users['id'] == uid]['following_list'])
    if len(ufl) > 0:
        users_following_lists.append(ufl)
    else:
        users_following_lists.append([])
print(time.time() - st , 'sec')

135.19613313674927 sec


In [10]:
len(users_following_lists)
users_following_lists[0][0]

5000

- r2
- author_lists
- class_lists
- magainze_lists
- unique_author

In [ ]:
# 20190207 이후에 읽은 글들로 앞으로 읽을 글들을 추천

In [8]:
with open(directory + 'rasby_devuser_100.csv',encoding='utf-8') as f:
    r1 = pd.read_csv(f)

r1 = r1.drop('Unnamed: 0',1)
r1['read_dt']  = r1['read_dt'].map(lambda x : x.replace('-',''))
a = '20190207' # 이것도 hyp 
r2 = r1[r1['read_dt'] >= a]

def int_class(x):
    if x == '5%':
        result = 5
    elif x == '10%':
        result = 10
    elif x == '25%':
        result = 25
    elif x == '50%':
        result = 50
    elif x == '75%':
        result = 75
    else:
        result = 100
    return result

In [14]:
r2[:2]

,read_dt,user_id,article_id,title,sub_title,author_id,reg_dt,type,display_url,keyword_list,magazine_id,off_day,read_cnt,class
0,20190212,#d6866a498157771069fdf15361cb012b,@april_78,"주택청약, 저는 언제쯤 제 집을...?",NaN,@april,2019-02-06,매거진,https://brunch.co.kr/@april/78,"['주택청약통장', '주택청약', '청약통장']",42262.0,6.0,300.0,5%
1,20190212,#d6866a498157771069fdf15361cb012b,@noonsil_12,틀린 게 아니고 다른 겁니다.,주부의 눈으로 본 태국이 다른 점,@noonsil,2019-02-08,매거진,https://brunch.co.kr/@noonsil/12,"['에어비앤비', '치앙마이', '파타야']",42485.0,4.0,703.0,5%


In [ ]:
# 평균 중복소비가 20건
# 평가 데이터에는 중복 소비가 없음
# 따라서 중복 소비 제거하고 최신 동향으로 추천하는 식 ㄱㄱ

In [9]:
# 읽은 구독자 list 도 만들기
# 개인 or magazine
# 이부분은 나중에 test_users_list , 5000 , test_user에 맞게 수정해주어야함
author_lists = []
class_lists = []
magazine_lists = []
unique_author = []
for i in tqdm(range(3000),mininterval = 1): ### final 시 수정
    recent_read_raw = r2[r2['user_id'] == dev_users_list[i]]  
    recent_read_raw = recent_read_raw.sort_values(by = ['read_dt'],ascending = False)
    recent_read_raw = recent_read_raw.drop_duplicates(['article_id'])
    
    len_recent_read_raw = len(recent_read_raw)
    
    # class mean
    class_sum = 0
    magazine_sum = 0
    # 최신에 읽은 작가들 정보 모으기
    author_list = []
    author_cnt = []
    for j in range(len_recent_read_raw):
        # class mean
        rrr = recent_read_raw[j:j+1]
        class_int = int_class(list(rrr['class'])[0])
        class_sum += class_int
        # magainze mean
        magazine_type = list(rrr['type'])[0]
        if(magazine_type == '매거진'):
            magazine_sum += 1
        # author list
        author = list(rrr['author_id'])[0]
        found = 0
        for k in range(len(author_list)):
            if author_list[k] == author:
                author_cnt[k] += 1
                found = 1
                break
        if(found == 0):
            author_list.append(author)
            author_cnt.append(1)

    author_list_dict = dict()
    for j in range(len(author_list)):
        author_list_dict[author_list[j]] = author_cnt[j]

    if(len_recent_read_raw != 0 ):
        class_mean = class_sum / len_recent_read_raw
        class_lists.append(class_mean)
        
        magazine_mean = magazine_sum / len_recent_read_raw
        magazine_lists.append(magazine_mean)
        
        #author_list_dict = sorted(author_list_dict.items(), key=(lambda x: x[1]), reverse = True) # 이러면 [ (), () ]형태
        author_lists.append(author_list_dict)
        
        unique_author.append(len(author_list))
    else:
        class_lists.append(5.0)
        magazine_lists.append(1.0)
        author_lists.append(dict())
        unique_author.append(0)



100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [01:07<00:00, 44.76it/s]


In [ ]:
# minimum 30은 차게 만들어보자

In [12]:
recent_cnt_lists = []
for i in range(len(dev_users_list)):
    author_and_cnt = author_lists[i]
    rcnt = 0
    for key, value in author_and_cnt.items():
        rcnt += value
    recent_cnt_lists.append(rcnt)

In [13]:
sum(recent_cnt_lists) / 3000

29.131666666666668

In [15]:
recent_cnt_lists[:15]

[57, 3, 9, 4, 37, 3, 7, 8, 49, 5, 4, 38, 7, 53, 25]

In [ ]:
# 아래 얘네들 키 안 맞음

In [46]:
# 최신에 읽은 글들에서 구독자들이 있는지 확인
# 없는 구독자는 이때까지 읽은 글을 검사해서 최신 동향이 아닌 구독자는 빼면 더 좋을 듯 하긴하네
diff_fa = []
diff_fa_cnt=[]
following_cnt =[]
diff_cnt = 0
for i in range(3000):
    uid = dev_users_list[i]
    keys = set(list(author_lists[i].keys()))
    diff_cnt = 0
    for j in range(5000):
        if  uid == test_users_list[j]:
            i = j
            break
    if len(users_following_lists[i]) > 0:
        following_author = set(users_following_lists[i][0])
        following_cnt.append(len(following_author))
    else :
        following_author = set()
        following_cnt.append(len(following_author))
    diff_fa.append(list(following_author - keys))
    diff_cnt += len(list(following_author - keys))
    diff_fa_cnt.append(diff_cnt)


In [50]:
following_cnt[:20]

[2, 12, 2, 1, 46, 4, 2, 9, 30, 3, 3, 19, 27, 40, 10, 23, 4, 2, 76, 31]

In [51]:
diff_fa_cnt[:20]

[1, 10, 0, 0, 35, 1, 0, 6, 20, 3, 2, 18, 24, 33, 4, 18, 2, 2, 71, 27]

In [54]:
diff_fa[15]

['@2jaemyung',
 '@brunch',
 '@nareun',
 '@pomnyun',
 '@realestate',
 '@simplebean33',
 '@chojeremy',
 '@boot0715',
 '@jaeyunchoi',
 '@shanghaiesther',
 '@isaacyouspace',
 '@ssj8201',
 '@seoyoungcla',
 '@pedkang01',
 '@stillalive31',
 '@kimpyogo',
 '@ellieyang47uu',
 '@novas99']

In [27]:
set(users_following_lists[1][0])

{'@brunch', '@heat0508', '@jhj3211', '@ohuk2011', '@sssfriend', '@syshine7'}

In [47]:
sum(diff_fa_cnt) / 3000

21.937666666666665

In [ ]:
author_lists 
class_lists 
magazine_lists 
unique_author 

# SIM 업데이트 버전

In [10]:
article_similarity = np.load(directory+'article_similarity_recent_t.npy')
article_similarity = article_similarity.tolist()
with open(directory + 'word_to_id_recent.json') as f:
    word_to_id  = json.load(f)

with open(directory + 'id_to_word_recent.json') as f:
    id_to_word = json.load(f)

print(len(article_similarity))
print(len(word_to_id))
print(len(id_to_word))

165110
486427
165110


# Dev User

# 좀더 신중히 다뤄 보도록 하자

In [ ]:
# FEB_top_30419 , 비교 필요

In [11]:
# read_cnt >= 10 만 자른것
FEB_top_30419 = FEB_top_58647[:30419]
FEB_top_30419_list = FEB_top_30419['article_id'].tolist()
len(FEB_top_30419)

30419

In [ ]:
- [조건]
    - 체크안된 구독자들 가져오는 맥시멈 아티클 pd 를 10개로 제한 (그럴 필요는 없겠는데? 그냥 평균 21개 가져오는데?)
    - 구독자 정보 추가할 때 제한 없앰. (5개 정도로 해도 되려나?) 일단 나오는거 봐서 판단하기
    - sim 추가하는게 좋음
    - 나머지는 그대로
    - 첫번째 두번째는 sim 과 sim 붙이지 않는 것만 차이있을 뿐 나머지는 다 똑같음 (sim 이냐 recent냐! 그것이 문제로다!)

In [60]:
#hyper parameter
hyp_author = 50
hyp_recent_article_len = 10
hyp_top_k = 10
hyp_read_cnt = 10
hyp_sim = 70

isdev = 1

hyp = str(hyp_author) + '_' +str(hyp_recent_article_len)+'_'+str(hyp_top_k)+'_'+ str(hyp_read_cnt)+'_'+str(hyp_sim)+'_'+'DEV_FINAL_1'

try:
    if not(os.path.isdir(directory + hyp )):
        os.makedirs(os.path.join(directory + hyp))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
            
countt = 0
cntttt = 0

after_author = []
after_author2 = []
siml = []
after_following = []
ulfs = []
after_sim = []
with open(directory + hyp +'/recommend' + '.txt','w') as file:
    for i in tqdm(range(len(dev_users_list)),mininterval=10):
        
        recommend_all=[]
        uid = dev_users_list[i]
        aid = i # author index 
        
        for j in range(5000):
            if uid == test_users_list[j]:
                i = j
                break
        
        # 현재 유저가 읽은 following list
        ufl = users_following_lists[i]
        if(len(ufl) > 0):
            ufl  = ufl[0]
            
        # 현재 유저가 읽은 모든 글 , 
        r = article_seen_by_testuser[str(i)]
        # 최신에 읽은 글들이 사라지지 않게 뒤집고 중복 제거하기
        r = list(reversed(r))
        s = []
        
        for a in range(len(r)):
            if r[a] not in s:
                s.append(r[a])
        r = list(reversed(s))

        # 최신 읽은 작가 정보를 업데이트
        # type(author_and_cnt) = dict 임
        # 또 하나 바꿔볼 방법은 구독자 
        author_and_cnt = author_lists[aid]
        if hyp_author > 0:
            
            # 일단 recent 2주 top 58647 에서
            recommend_author = []
            cnt_author = 0            
            author_consider_set = set()
            
            for key, value in author_and_cnt.items(): 
                author = key
                cnt = value
                author_consider_set.add(key)
                # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(cnt_1 >= cnt): break
                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_1 +=1
                
                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(cnt_2 >= cnt): break
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_2 +=1
                            
                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(cnt_3 >= cnt): break
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_3 +=1
            recommend_author = recommend_author[:hyp_author] # 최신 읽은 글 경향 자르기
                
            for artic in recommend_author:
                if artic not in recommend_all:
                    recommend_all.append(artic)
        after_author.append(len(recommend_all))
        
        if(len(recommend_all) > 0): # 최신에 읽은 글이 있다는 소리고
            iter_1_cnt = 0
            while(len(recommend_all) < 80) : # 최신의 글을 고려해서 최대 30~ 개 까지만 넣어주겠다 # 구독작가 && 최신 읽은 작가
                # 혹시 읽은 작가 중에 2-14 - 3-14 까지 쓴 글이 없을 수 있으므로
                if(iter_1_cnt >= 30) : break
                iter_1_cnt += 1
                recommend_author = []
                cnt_author = 0            
                
                for key, value in author_and_cnt.items(): 
                    author = key
                    if(len(recommend_all) >= 100): break
                    if author in ufl:

                        cnt = value
                        # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                        author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                        cnt_1 = 0
                        for j in range(len(author_all_article_1)):
                            if(cnt_1 >= cnt): break
                            artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                            if artic not in r:
                                if artic not in recommend_all:
                                    recommend_all.append(artic)
                                    cnt_1 +=1

                        #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                        author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                        cnt_2 = 0
                        for j in range(len(author_all_article_2)):
                            if(cnt_2 >= cnt): break
                            artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                            if artic not in r:
                                if artic not in recommend_all:
                                    recommend_all.append(artic)
                                    cnt_2 +=1

                        #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                        author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                        cnt_3 = 0
                        for j in range(len(author_all_article_3)):
                            if(cnt_3 >= cnt): break
                            artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                            if artic not in r:
                                if artic not in recommend_all:
                                    recommend_all.append(artic)
                                    cnt_3 +=1
                
        
        

         # 100 개 이상 추천될 수 있으니 자르기 | hyper parameter 만큼 자르기
        if len(recommend_all) >= 100:
            recommend_all = recommend_all[:100]
        
        
        after_author2.append(len(recommend_all))
        
        # 최신에 읽지 않은 구독자의 글들도 넣어보자
        # recent때 활용하지 않은 구독자 정보를 이용한다
        ufl_not_yet = []
        for t in range(len(ufl)):
            if ufl[t] not in author_consider_set:
                ufl_not_yet.append(ufl[t])
        
        ufls_i = 0
        if len(ufl_not_yet) > 0:

            for author in ufl_not_yet: #
                if(len(recommend_all) >= 100): break
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # top 50000 or to 10000 으로 바꿔서 비
                # 너무 많은 글들이 있으면 10개로 자르자
                if(len(author_all_article_1) > 10 ): 
                    author_all_article_1 = author_all_article_1[:10]
                for j in range(len(author_all_article_1)):
                    if(len(recommend_all) >= 100): break #이것도 조정 필요

                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            ufls_i+=1


                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                if(len(author_all_article_2) > 10 ): 
                    author_all_article_2 = author_all_article_2[:10]
                for j in range(len(author_all_article_2)):
                    if(len(recommend_all) >= 100): break #얘도 조정 필요하고
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            ufls_i+=1

                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                if(len(author_all_article_3) > 10 ): 
                    author_all_article_3 = author_all_article_3[:10]
                for j in range(len(author_all_article_3)):
                    if(len(recommend_all) >= 100): break 
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            ufls_i+=1
            

        
        ulfs.append(ufls_i)
        after_following.append(len(recommend_all))
        
        # 최신에 본 글 위주로 similarity 조사
        
        if(len(r) >= hyp_read_cnt):
            wtil = list(word_to_id.keys())
            countt +=1
            if(hyp_recent_article_len > len(r)):
                hyp_recent_article_len = len(r)
            r = r[-hyp_recent_article_len:]
            r = list(reversed(r))
            siml_cnt = 0
            for j in range(len(r)):
                if(siml_cnt>= hyp_sim): break
                if(r[j] not in wtil):
                    r[j] = 'UNK'
                s = article_similarity[word_to_id[r[j]]]
                sim_list = s[:hyp_top_k]
                for k in range(len(sim_list)):
                    if len(recommend_all)>=100: break
                            
                    if sim_list[k] not in recommend_all:
                        recommend_all.append(sim_list[k])
                        siml_cnt+=1
                        if(siml_cnt>= hyp_sim): break
            siml.append(siml_cnt)
            if len(recommend_all)>=100:
                recommend_all = recommend_all[:100]
        
        
        after_sim.append(len(recommend_all))
        
        if(len(recommend_all) >= 100):
            recommend_all = recommend_all[:100]
        else:
            # 남으면 FEB_top 중 아무거나 뽑아서 주고
            # following list 있으면 여기에 활용해도 좋을듯
            cntttt+=1
            
            
            for x in FEB_top_30419_list:
                if x not in recommend_all:
                    recommend_all.append(x)
                    if(len(recommend_all) == 100): break
                        
        # 다시 한 번 더 점검
        if(len(recommend_all) > 100):
            recommend_all = recommend_all[:100]
        

        recommend_normal = ''
        for x in recommend_all:
            recommend_normal += ' '+ x

        file.write(uid + recommend_normal)
        file.write('\n')


100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [18:19<00:00,  2.73it/s]


In [57]:
# 70 / 30 / UF / sim = 70
print(sum(after_author) / 3000)
print(sum(after_author2) / 3000)
print(sum(after_following) /3000)
print(sum(after_sim) / 3000)

print()
print(sum(ulfs) /3000)
print(sum(siml) /3000)
# 0.073169	0.175026	9.087112

40.035
43.091
71.96233333333333
90.26366666666667

28.871333333333332


In [59]:
# 80 / 80 / UF / sim = 70
print(sum(after_author) / 3000)
print(sum(after_author2) / 3000)
print(sum(after_following) /3000)
print(sum(after_sim) / 3000)

print()
print(sum(ulfs) /3000) # 최근에 읽지 않은 구독자로 채워지는 부분
print(sum(siml) /3000) # sim 으로 채워지는 부분
print(cntttt)          # most_recent_top 으로 채워지는 사람들 수

# 0.073070	0.174045	9.069491

42.647666666666666
58.666333333333334
78.57833333333333
90.852

19.912
12.273666666666667
515


In [61]:
# 50 / 80 / UF / sim = 70
print(sum(after_author) / 3000)
print(sum(after_author2) / 3000)
print(sum(after_following) /3000)
print(sum(after_sim) / 3000)

print()
print(sum(ulfs) /3000) # 최근에 읽지 않은 구독자로 채워지는 부분
print(sum(siml) /3000) # sim 으로 채워지는 부분
print(cntttt)          # most_recent_top 으로 채워지는 사람들 수

32.875
57.93066666666667
77.427
90.852

19.496333333333332
13.425
515


# CURRENT BEST

In [62]:
#hyper parameter
hyp_author = 50
hyp_recent_article_len = 10
hyp_top_k = 10
hyp_read_cnt = 10
hyp_sim = 70

isdev = 1

hyp = str(hyp_author) + '_' + '_'+str(hyp_sim)+'_'+'DEV_FINAL_1'

try:
    if not(os.path.isdir(directory + hyp )):
        os.makedirs(os.path.join(directory + hyp))
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!!!!!")
        raise
            
countt = 0
cntttt = 0

after_author = []
after_author2 = []
siml = []
after_following = []
after_recent = []
ulfs = []
after_sim = []
with open(directory + hyp +'/recommend' + '.txt','w') as file:
    for i in tqdm(range(len(dev_users_list)),mininterval=10):
        
        recommend_all=[]
        uid = dev_users_list[i]
        aid = i # author index 
        
        for j in range(5000):
            if uid == test_users_list[j]:
                i = j
                break
        
        # 현재 유저가 읽은 following list
        ufl = users_following_lists[i]
        if(len(ufl) > 0):
            ufl  = ufl[0]
            
        # 현재 유저가 읽은 모든 글 , 
        r = article_seen_by_testuser[str(i)]
        # 최신에 읽은 글들이 사라지지 않게 뒤집고 중복 제거하기
        r = list(reversed(r))
        s = []
        
        for a in range(len(r)):
            if r[a] not in s:
                s.append(r[a])
        r = list(reversed(s))

        # 최신에 읽은 사람들 중 구독자가 있는 것부터 먼저 띄우기
        # 최신 && 구독
        author_and_cnt = author_lists[aid]
        if hyp_author > 0:
            
            # 일단 recent 2주 top 58647 에서
            recommend_author = []
            cnt_author = 0            
            author_consider_set = set()
            
            for key, value in author_and_cnt.items(): 
                author = key
                if author not in ufl: continue
                
                cnt = value
                author_consider_set.add(key)
                # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(cnt_1 >= cnt): break
                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_1 +=1
                
                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(cnt_2 >= cnt): break
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_2 +=1
                            
                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(cnt_3 >= cnt): break
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_3 +=1
                
            for artic in recommend_author:
                if artic not in recommend_all:
                    recommend_all.append(artic)
                    
        if(len(recommend_all) > 100):
            recommend_all = recommend_all[:100]
        
        after_author.append(len(recommend_all))
        
        # 최신 && 구독 minimum > 50 개 채우기
        if(len(recommend_all) > 0): # 최신에 읽은 글이 있다는 소리고
            iter_1_cnt = 0
            while(len(recommend_all) < 50) : # 최신의 글을 고려해서 최대 30~ 개 까지만 넣어주겠다 # 구독작가 && 최신 읽은 작가
                # 혹시 읽은 작가 중에 2-14 - 3-14 까지 쓴 글이 없을 수 있으므로, 적게 뽑힐 수 있으므로 60까지는 구독자가 읽은 글들로 채우기
                if(iter_1_cnt >= 30) : break
                iter_1_cnt += 1
                recommend_author = []
                cnt_author = 0            
                
                for key, value in author_and_cnt.items(): 
                    author = key
                    if(len(recommend_all) >= 100): break
                    if author in ufl:

                        cnt = value
                        # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                        author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                        cnt_1 = 0
                        for j in range(len(author_all_article_1)):
                            if(cnt_1 >= cnt): break
                            artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                            if artic not in r:
                                if artic not in recommend_all:
                                    recommend_all.append(artic)
                                    cnt_1 +=1

                        #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                        author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                        cnt_2 = 0
                        for j in range(len(author_all_article_2)):
                            if(cnt_2 >= cnt): break
                            artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                            if artic not in r:
                                if artic not in recommend_all:
                                    recommend_all.append(artic)
                                    cnt_2 +=1

                        #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                        author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                        cnt_3 = 0
                        for j in range(len(author_all_article_3)):
                            if(cnt_3 >= cnt): break
                            artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                            if artic not in r:
                                if artic not in recommend_all:
                                    recommend_all.append(artic)
                                    cnt_3 +=1
                
        
        
        after_author2.append(len(recommend_all))
        
        
        

        # 최근 안 읽은 && 구독자
        ufl_not_yet = []
        for t in range(len(ufl)):
            if ufl[t] not in author_consider_set:
                ufl_not_yet.append(ufl[t])
        
        ufls_i = 0
        if len(ufl_not_yet) > 0:

            for author in ufl_not_yet: #
                if(len(recommend_all) >= 100): break
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # top 50000 or to 10000 으로 바꿔서 비
                # 너무 많은 글들이 있으면 10개로 자르자
                if(len(author_all_article_1) > 10 ): 
                    author_all_article_1 = author_all_article_1[:10]
                for j in range(len(author_all_article_1)):
                    if(len(recommend_all) >= 100): break #이것도 조정 필요

                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            ufls_i+=1


                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                if(len(author_all_article_2) > 10 ): 
                    author_all_article_2 = author_all_article_2[:10]
                for j in range(len(author_all_article_2)):
                    if(len(recommend_all) >= 100): break #얘도 조정 필요하고
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            ufls_i+=1

                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                if(len(author_all_article_3) > 10 ): 
                    author_all_article_3 = author_all_article_3[:10]
                for j in range(len(author_all_article_3)):
                    if(len(recommend_all) >= 100): break 
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_all:
                            recommend_all.append(artic)
                            ufls_i+=1
            
        if(len(recommend_all) > 100):
            recommend_all = recommend_all[:100]
            
        ulfs.append(ufls_i)
        after_following.append(len(recommend_all))
        
        # 최신에 읽음 && 구독자 아님
        if hyp_author > 0:
            
            # 일단 recent 2주 top 58647 에서
            recommend_author = []
            cnt_author = 0            
            author_consider_set = set()
            
            for key, value in author_and_cnt.items(): 
                author = key
                if author in ufl: continue
                
                cnt = value
                author_consider_set.add(key)
                # 2월 14일 ~ 3월 1일간 인기글 30419개 중에 있으면 뽑기
                author_all_article_1 = FEB_top_30419[FEB_top_30419['author_id'] == author].sort_values(by='reg_dt',ascending=False) # 최신에 작성한 글들의 인기순이겠지 # 더 줄여봐도 될 듯
                cnt_1 = 0
                for j in range(len(author_all_article_1)):
                    if(cnt_1 >= cnt): break
                    artic = list(author_all_article_1[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_1 +=1
                
                #  3월 1일 ~ 3월 7일에 쓰여진 글 있으면 append
                author_all_article_2 = MAR_article_1[MAR_article_1['author_id'] == author]
                cnt_2 = 0
                for j in range(len(author_all_article_2)):
                    if(cnt_2 >= cnt): break
                    artic = list(author_all_article_2[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_2 +=1
                            
                #  3월 7일 ~ 3월 14일에 쓰여진 글 있으면 append
                author_all_article_3 = MAR_article_2[MAR_article_2['author_id'] == author]
                cnt_3 = 0
                for j in range(len(author_all_article_3)):
                    if(cnt_3 >= cnt): break
                    artic = list(author_all_article_3[j:j+1]['article_id'])[0]
                    if artic not in r:
                        if artic not in recommend_author:
                            recommend_author.append(artic)
                            cnt_3 +=1
            
                
            for artic in recommend_author:
                if artic not in recommend_all:
                    recommend_all.append(artic)
        
        

        # 100 개 이상 추천될 수 있으니 자르기 | hyper parameter 만큼 자르기
        if len(recommend_all) >= 100:
            recommend_all = recommend_all[:100]
        
        after_recent.append(len(recommend_all))
        
        
        
        

        # 최신에 본 글 위주로 similarity 조사
        # 최신에 본 글 + similarity (전체 유저)
        if(len(r) >= hyp_read_cnt):
            wtil = list(word_to_id.keys())
            countt +=1
            if(hyp_recent_article_len > len(r)):
                hyp_recent_article_len = len(r)
            r = r[-hyp_recent_article_len:]
            r = list(reversed(r))
            siml_cnt = 0
            for j in range(len(r)):
                if(siml_cnt>= hyp_sim): break
                if(r[j] not in wtil):
                    r[j] = 'UNK'
                s = article_similarity[word_to_id[r[j]]]
                sim_list = s[:hyp_top_k]
                for k in range(len(sim_list)):
                    if len(recommend_all)>=100: break
                            
                    if sim_list[k] not in recommend_all:
                        recommend_all.append(sim_list[k])
                        siml_cnt+=1
                        if(siml_cnt>= hyp_sim): break
            siml.append(siml_cnt)
            if len(recommend_all)>=100:
                recommend_all = recommend_all[:100]
        
        
        after_sim.append(len(recommend_all))
        
        if(len(recommend_all) >= 100):
            recommend_all = recommend_all[:100]
        else:
            # 남으면 FEB_top 중 아무거나 뽑아서 주고
            # following list 있으면 여기에 활용해도 좋을듯
            cntttt+=1
            
            # 나머지는 그냥 2월 14일 - 3월 1일까지 글들 중 top 뽑기
            for x in FEB_top_30419_list:
                if x not in recommend_all:
                    recommend_all.append(x)
                    if(len(recommend_all) == 100): break
                        
        # 다시 한 번 더 점검
        if(len(recommend_all) > 100):
            recommend_all = recommend_all[:100]
        

        recommend_normal = ''
        for x in recommend_all:
            recommend_normal += ' '+ x

        file.write(uid + recommend_normal)
        file.write('\n')


100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [17:13<00:00,  2.90it/s]


In [63]:
# 구독+최근 / 50(구독+최근) / UF / recent / sim = 70 / 나머지
print(sum(after_author) / 3000)
print(sum(after_author2) / 3000)
print(sum(after_following) /3000)
print(sum(after_recent)/3000)
print(sum(after_sim) / 3000)

print()
print(sum(ulfs) /3000) # 최근에 읽지 않은 구독자로 채워지는 부분
print(sum(siml) /3000) # sim 으로 채워지는 부분
print(cntttt)          # most_recent_top 으로 채워지는 사람들 수

14.686666666666667
33.666
64.03733333333334
78.55633333333333
90.723

30.371333333333332
12.166666666666666
522


In [68]:
after_author[:20]

[3, 9, 10, 3, 30, 5, 9, 13, 39, 0, 3, 2, 9, 29, 30, 26, 6, 0, 13, 10]

In [69]:
after_author2[:20]

[3, 50, 10, 51, 66, 48, 39, 24, 63, 0, 51, 32, 35, 54, 61, 67, 36, 0, 51, 40]

In [71]:
after_following[:15]

[14, 100, 10, 51, 100, 48, 39, 46, 88, 19, 72, 100, 100, 100, 75]

In [72]:
after_recent[:15]

[100, 100, 13, 56, 100, 48, 42, 55, 100, 24, 73, 100, 100, 100, 89]

In [77]:
after_sim[:15]

[100, 100, 83, 100, 100, 48, 100, 100, 100, 24, 73, 100, 100, 100, 100]

In [80]:
cnt=0
sum_cnt=0
for i in range(3000):
    if (after_sim[i] != 100):
        cnt +=1
        sum_cnt+= after_sim[i]

In [81]:
sum_cnt/cnt

46.68390804597701